<a href="https://colab.research.google.com/github/newmantic/red_black_tree/blob/main/red_black_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
class Node:
    def __init__(self, key, color='red'):
        self.key = key
        self.color = color
        self.left = None
        self.right = None
        self.parent = None

class RedBlackTree:
    def __init__(self):
        self.TNULL = Node(0, color='black')  # Sentinel node for leaves (NIL)
        self.root = self.TNULL

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, y):
        x = y.left
        y.left = x.right
        if x.right != self.TNULL:
            x.right.parent = y
        x.parent = y.parent
        if y.parent is None:
            self.root = x
        elif y == y.parent.right:
            y.parent.right = x
        else:
            y.parent.left = x
        x.right = y
        y.parent = x

    def insert_fixup(self, node):
        while node.parent and node.parent.color == 'red':
            if node.parent == node.parent.parent.left:
                uncle = node.parent.parent.right
                if uncle.color == 'red':  # Case 1: Uncle is red
                    node.parent.color = 'black'
                    uncle.color = 'black'
                    node.parent.parent.color = 'red'
                    node = node.parent.parent
                else:
                    if node == node.parent.right:  # Case 2: Node is right child
                        node = node.parent
                        self.left_rotate(node)
                    node.parent.color = 'black'  # Case 3: Node is left child
                    node.parent.parent.color = 'red'
                    self.right_rotate(node.parent.parent)
            else:
                uncle = node.parent.parent.left
                if uncle.color == 'red':  # Case 1: Uncle is red
                    node.parent.color = 'black'
                    uncle.color = 'black'
                    node.parent.parent.color = 'red'
                    node = node.parent.parent
                else:
                    if node == node.parent.left:  # Case 2: Node is left child
                        node = node.parent
                        self.right_rotate(node)
                    node.parent.color = 'black'  # Case 3: Node is right child
                    node.parent.parent.color = 'red'
                    self.left_rotate(node.parent.parent)
        self.root.color = 'black'

    def insert(self, key):
        node = Node(key)
        node.left = self.TNULL
        node.right = self.TNULL

        parent = None
        current = self.root

        while current != self.TNULL:
            parent = current
            if node.key < current.key:
                current = current.left
            else:
                current = current.right

        node.parent = parent
        if parent is None:
            self.root = node
        elif node.key < parent.key:
            parent.left = node
        else:
            parent.right = node

        node.color = 'red'
        self.insert_fixup(node)

    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    def delete_fixup(self, node):
        while node != self.root and node.color == 'black':
            if node == node.parent.left:
                sibling = node.parent.right
                if sibling.color == 'red':  # Case 1: Sibling is red
                    sibling.color = 'black'
                    node.parent.color = 'red'
                    self.left_rotate(node.parent)
                    sibling = node.parent.right
                if sibling.left.color == 'black' and sibling.right.color == 'black':  # Case 2: Sibling has two black children
                    sibling.color = 'red'
                    node = node.parent
                else:
                    if sibling.right.color == 'black':  # Case 3: Sibling's right child is black
                        sibling.left.color = 'black'
                        sibling.color = 'red'
                        self.right_rotate(sibling)
                        sibling = node.parent.right
                    sibling.color = node.parent.color  # Case 4: Sibling's right child is red
                    node.parent.color = 'black'
                    sibling.right.color = 'black'
                    self.left_rotate(node.parent)
                    node = self.root
            else:
                sibling = node.parent.left
                if sibling.color == 'red':  # Case 1: Sibling is red
                    sibling.color = 'black'
                    node.parent.color = 'red'
                    self.right_rotate(node.parent)
                    sibling = node.parent.left
                if sibling.right.color == 'black' and sibling.left.color == 'black':  # Case 2: Sibling has two black children
                    sibling.color = 'red'
                    node = node.parent
                else:
                    if sibling.left.color == 'black':  # Case 3: Sibling's left child is black
                        sibling.right.color = 'black'
                        sibling.color = 'red'
                        self.left_rotate(sibling)
                        sibling = node.parent.left
                    sibling.color = node.parent.color  # Case 4: Sibling's left child is red
                    node.parent.color = 'black'
                    sibling.left.color = 'black'
                    self.right_rotate(node.parent)
                    node = self.root
        node.color = 'black'

    def delete(self, key):
        node = self.root
        z = self.TNULL
        while node != self.TNULL:
            if node.key == key:
                z = node
            if node.key < key:
                node = node.right
            else:
                node = node.left

        if z == self.TNULL:
            print("Key not found in the tree")
            return

        y = z
        y_original_color = y.color
        if z.left == self.TNULL:
            x = z.right
            self._transplant(z, z.right)
        elif z.right == self.TNULL:
            x = z.left
            self._transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self._transplant(y, y.right)
                y.right = z.right
                y.right.parent = y
            self._transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        if y_original_color == 'black':
            self.delete_fixup(x)

    def _transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def inorder_traversal(self, node):
        if node == self.TNULL:
            return []
        return self.inorder_traversal(node.left) + [node.key] + self.inorder_traversal(node.right)


In [5]:
# Example 1: Insertion
rb_tree = RedBlackTree()
keys = [20, 15, 25, 10, 5, 1]
for key in keys:
    rb_tree.insert(key)

# The in-order traversal should return a sorted sequence of inserted keys
print("Inorder traversal after insertion:", rb_tree.inorder_traversal(rb_tree.root))  # Should be [1, 5, 10, 15, 20, 25]

# Example 2: Deletion
rb_tree.delete(10)
print("Inorder traversal after deleting 10:", rb_tree.inorder_traversal(rb_tree.root))  # Should be [1, 5, 15, 20, 25]

rb_tree.delete(1)
print("Inorder traversal after deleting 1:", rb_tree.inorder_traversal(rb_tree.root))

Inorder traversal after insertion: [1, 5, 10, 15, 20, 25]
Inorder traversal after deleting 10: [1, 5, 15, 20, 25]
Inorder traversal after deleting 1: [5, 15, 20, 25]
